In [8]:
load("utils.sage")

# Pick parameters
p = 419
L = [2,3]

# Get L isogeny graph
G = get_L_graph(p, L)
print("Graph on " + str(len(G.vertices())) + " vertices and " + str(len(G.edges())) + " edges.")

# Get Laplacian matrix
L = G.laplacian_matrix()

# Get eigenvalues/vectors
#     Sage has a method     - sometimes slower, but more accurate result
#     Numpy performs better - sometimes faster, but less accurate results  (i.e. vertices might be ordered slightly differently in sweep cuts)
use_numpy_over_sage = False
if use_numpy_over_sage:
    import numpy as np
    from numpy.linalg import eig
    a = np.array(L.rows())
    w,v=eig(a)
    es = list(zip(w, v))
else:
    es_nonflat = L.eigenvectors_right()
    es = []
    for e in es_nonflat:
        for vec in e[1]:
            es.append((e[0], vec))
# es is now a list of tuples [(eigenvalue, eigenvector), (eigenvalue, eigenvector), ...]

# Sort to find the 2nd largest eigenvalue, and its eigenvector, the fiedler vector
def first_entry(e):
    return e[0]
es.sort(key=first_entry, reverse=True)
lambda2 = es[1][0]
fiedler_vector = list(es[1][1])
print("Second largest eigenvalue: " + str(lambda2))
print("Fiedler vector (first 10 entries):")
print([float(v) for v in fiedler_vector[:10]])

# Order vertices by their eigenvector entry
vs_and_es = list(zip(fiedler_vector, G.vertices()))    # list of tuples [(eigenvector_entry, vertex_label), (eigenvector_entry, vertex_label), ...]
vs_and_es.sort(key=first_entry, reverse=True)

# Iterate over sweep cuts
best_cut_so_far = None
best_edge_expansion_so_far = None
for i in range(1, len(G.vertices())):
    Ci = [v[1] for v in vs_and_es[:i]]
    Ci_comp = vertex_complement(G, Ci)
    # Find sweep cut with minimal edge expansion, or more precisely  max( h(C_i) ,  h(V not C_i) )
    Ci_expansion = edge_expansion(G, Ci)
    Ci_comp_expansion = edge_expansion(G, Ci_comp)
    exp_max = max(Ci_expansion, Ci_comp_expansion)
    if best_edge_expansion_so_far == None or exp_max < best_edge_expansion_so_far:
        best_edge_expansion_so_far = exp_max
        best_cut_so_far = Ci

print("")
print("Minimial edge expansion: " + str(best_edge_expansion_so_far))
print("Size of resulting cut: " + str(len(best_cut_so_far)))

Graph on 36 vertices and 252 edges.
Second largest eigenvalue: 11.172240022640950?
Fiedler vector (first 10 entries):
[1.0, 0.1746998625382865, 0.03399357970293326, 0.09582971918469063, -1.6539794293125492, -0.16983484813517613, -0.3130143089536399, 0.038832693624163986, -0.513019312222551, -0.27373466156792625]

Minimial edge expansion: 0.4603174603174603
Size of resulting cut: 18


We've combined this into a single function for convenience.

In [11]:
expansion, cut = fiedler(G, use_numpy_over_sage=False)
expansion

0.4603174603174603